In [48]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import h5py
from scipy.stats import entropy
from scipy.spatial.distance import pdist, squareform
import scipy.cluster.hierarchy as _sch
from activ import load_data
from activ.readfile import load_preprocessed, TrackTBIFile, encode, impute
import pandas as pd
from math import ceil
from matplotlib import cm
from sklearn import preprocessing
from collections import Counter
from activ.data_normalization import data_normalization

In [439]:
subdata = load_data()
subdata_oc = subdata.outcomes
subdata_bm = subdata.biomarkers
oc_features = subdata.outcome_features
bm_features = subdata.biomarker_features
sub_features = np.hstack((subdata.biomarker_features, subdata.outcome_features))
ids = subdata.patient_ids

In [4]:
fulldata = load_preprocessed()

In [5]:
gose12 = fulldata[np.logical_or(fulldata.GOSE_OverallScore6M == 1.0, fulldata.GOSE_OverallScore6M == 2.0)]

In [7]:
gose12_ids = gose12.index

In [8]:
# one-hot encoding oc
gose12['GOSE_Reponder6M_Patient.alone'] = np.zeros(29)
gose12['GOSE_Reponder6M_Patient.plus.relative'] = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# one-hot encoding bm

gose12['cause_Assault'] = [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0]
gose12['cause_Fall'] = [1,1,1,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0]
gose12['cause_MV.car.'] = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]
gose12['cause_MV.cyclist.ped.'] = [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
gose12['cause_MV.motorcycle.'] = np.zeros(29)
gose12['cause_Other'] = np.zeros(29)
gose12['cause_Struck.By'] = np.zeros(29)
gose12['DispER_Discharge.Home'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose12['DispER_ICU.Admit'] = [0,1,0,1,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0]
gose12['DispER_Operating.Room'] = [0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0]
gose12['DispER_Stepdown.Admit'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0]
gose12['DispER_Ward Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0]
gose12['PatientTypeCoded_ED.Discharge'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose12['PatientTypeCoded_ICU.Admit'] = [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0]
gose12['PatientTypeCoded_Ward.Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0]
gose12['PresArrivalMethod_Ambulance'] = [1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1]
gose12['PresArrivalMethod_Helicopter'] = [0,1,0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
gose12['PresArrivalMethod_Walk.in'] = np.zeros(29)
gose12['RACE_3CAT_Black'] = np.zeros(29)
gose12['RACE_3CAT_Other'] = np.zeros(29)
gose12['RACE_3CAT_White'] = np.ones(29)
gose12['injurytype_Closed'] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1]
gose12['injurytype_Penetrating'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]

/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [10]:
keep_cols = np.asarray(list(set(sub_features).intersection(gose12.columns)))

In [12]:
gose12_oc_features = np.asarray(list(set(keep_cols).intersection(oc_features)))
gose12_bm_features = np.asarray(list(set(keep_cols).intersection(bm_features)))

In [13]:
drop_cols = list(set(gose12.columns).difference(keep_cols))

In [14]:
gose12.shape

(29, 943)

In [15]:
gose12 = gose12.drop(columns = drop_cols)

In [405]:
gose12_bm = gose12.drop(columns = gose12_oc_features)
gose12_oc = gose12.drop(columns = gose12_bm_features)

In [406]:
gose12_bm = gose12_bm[bm_features]

In [411]:
gose12_oc = gose12_oc[oc_features]

In [414]:
gose12_bm.shape, gose12_oc.shape

((29, 235), (29, 177))

In [20]:
pd.DataFrame(gose12_bm).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_bm.csv")
pd.DataFrame(gose12_oc).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_oc.csv")

In [584]:
subdata_bm = pd.DataFrame(subdata_bm, columns=bm_features)
subdata_oc = pd.DataFrame(subdata_oc, columns=oc_features)

In [585]:
for ii,f in enumerate(bm_features):
    if (1. in np.unique(subdata_bm[f])) and (2. in np.unique(subdata_bm[f])) and len(np.unique(subdata_bm[f]))==2:
        subdata_bm[f] = data_normalization(subdata_bm[f], 'positive')

In [586]:
for ii,f in enumerate(oc_features):
    if (1. in np.unique(subdata_oc[f])) and (2. in np.unique(subdata_oc[f])) and len(np.unique(subdata_oc[f]))==2:
        subdata_oc[f] = data_normalization(subdata_oc[f], 'positive')

In [ ]:
pd.DataFrame(gose12_bm).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_bm.csv")
pd.DataFrame(gose12_oc).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_oc.csv")

In [225]:
def encode(df, get_binvars=False):
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder, LabelBinarizer
    cols = dict()
    enc = LabelEncoder()
    lb = LabelBinarizer()
    binvars = dict()
    for colname, coltype in df.dtypes.items():
        if coltype == object:
            newcol = enc.fit_transform(df[colname].astype(str))
            cols[colname] = newcol
            binvars[colname] = enc.classes_
        else:
            cols[colname] = df[colname]
    newdf = pd.DataFrame(cols, index=df.index)
    if get_binvars:
        return newdf, binvars
    else:
        return newdf

In [561]:
gose12_bm_redo = pd.read_csv('/Users/ahyeon/Desktop/activ_misc/gose12_bm_redo.csv', header=None)

In [562]:
gose12_bm = gose12_bm_redo.iloc[1:,1:]

In [465]:
gose12_bm_redo_columns = gose12_bm_redo.iloc[0,1:].astype(str)

In [466]:
gose12_bm.columns = gose12_bm_redo_columns

In [470]:
gose12_bm_ordered = gose12_bm[bm_features]

In [563]:
redo_id = gose12_bm_redo.iloc[:,0][1:]

In [564]:
redo_id

1     BR-1043
2     BR-1078
3     PI-1014
4     PI-1020
5     PI-1046
6     PI-1051
7     PI-1052
8     PI-1053
9     PI-1061
10    PI-1065
11    PI-1091
12    PI-1098
13    PI-1113
14    PI-1131
15    PI-1137
16    PI-1145
17    PI-1161
18    PI-1175
19    SF-1009
20    SF-1073
21    SF-1118
22    SF-1129
23    SF-1130
24    SF-1162
25    SF-1168
26    SF-1211
27    SF-1267
28    SF-1271
29    SF-1316
Name: 0, dtype: object

In [312]:
age = gose12_bm_redo.iloc[:,1]

In [489]:
gose12_bm_ordered_encode = encode(gose12_bm_ordered.iloc[:,1:])

In [490]:
gose12_bm_ordered_encode

,Sex,RaceAmericanIndian,RaceAmAl,RaceAsian,RaceBlack,RaceHAPI,RaceWhite,RACE_3CAT_Black,RACE_3CAT_Other,RACE_3CAT_White,...,contusionparietalright,contusionparietalleft,contusiontemporalright,contusiontemporalleft,contusionmidbrainleft,ICHoccipitalright,MidlineShiftmm,cisternalcompressionSeverity,Fourthventricleshift,Thirdventricleobliteration
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,6,2,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,1,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,1,1,2
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,14,1,0,2
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,7,1,0,2
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,1,2
9,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,10,2,0,1


In [491]:
df = gose12_bm_ordered_encode.insert(0,'Age',age)

In [492]:
## final final BM 

gose12_bm_ordered_encode

,Age,Sex,RaceAmericanIndian,RaceAmAl,RaceAsian,RaceBlack,RaceHAPI,RaceWhite,RACE_3CAT_Black,RACE_3CAT_Other,...,contusionparietalright,contusionparietalleft,contusiontemporalright,contusiontemporalleft,contusionmidbrainleft,ICHoccipitalright,MidlineShiftmm,cisternalcompressionSeverity,Fourthventricleshift,Thirdventricleobliteration
1,82,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,6,2,0,1
2,59,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
3,44,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,1,1
4,63,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,26,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,1,1,2
6,63,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,14,1,0,2
7,46,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,7,1,0,2
8,62,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,1,2
9,74,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10,74,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,10,2,0,1


In [538]:
gose12_oc_redo = pd.read_csv('/Users/ahyeon/Desktop/activ_misc/gose12_oc_redo.csv', header=None)

In [539]:
gose12_oc = gose12_oc_redo.iloc[1:,1:]

In [540]:
gose12_oc_redo_columns = gose12_oc_redo.iloc[0,1:].astype(str)

In [541]:
gose12_oc.columns = gose12_oc_redo_columns

In [546]:
## final final oc
gose12_oc_ordered = gose12_oc[oc_features]

In [589]:
final_gose12_bm = np.vstack((subdata_bm, gose12_bm_ordered_encode))
final_gose12_oc = np.vstack((subdata_oc, gose12_oc_ordered))
final_ids = np.hstack((ids, redo_id))

In [590]:
final_gose12_bm = np.asarray(final_gose12_bm, dtype=float)
final_gose12_oc = np.asarray(final_gose12_oc, dtype=float)

In [596]:
final_gose12_bm.shape, final_gose12_oc.shape

((276, 235), (276, 177))

In [597]:
from sklearn.utils import shuffle
bm_shuffle, oc_shuffle, id_shuffle = shuffle(final_gose12_bm,final_gose12_oc,final_ids)

In [598]:
fname_shuffled = '/Users/ahyeon/Desktop/activ_misc/gose12_shuffled_final.h5'
TrackTBIFile.write(fname_shuffled, bm_shuffle, oc_shuffle, biomarker_features = bm_features, outcome_features=oc_features, patient_ids=id_shuffle)

In [361]:
path = '/Users/ahyeon/Desktop/activ_misc/gose12_shuffled_final.h5'
f = h5py.File(path, mode='r')
try:
    bm = f['biomarkers'][:]
    oc = f['outcomes'][:]
    bm_features = f['biomarker_features'][:]
    oc_features = f['outcome_features'][:]
finally:
    f.close()

In [367]:
bm_features = bm_features.astype(str)

In [376]:
for ii,f in enumerate(bm_features):
    if 'cause' in f:
        print(ii,f)

94 cause_Assault
95 cause_Fall
96 cause_MV.car.
97 cause_MV.cyclist.ped.
98 cause_MV.motorcycle.
99 cause_Other
100 cause_Struck.By


In [389]:
Counter(bm[:,95])

Counter({1.0: 79,
         0.0: 171,
         5.0: 10,
         2.0: 2,
         7.0: 3,
         4.0: 5,
         6.0: 2,
         3.0: 3,
         8.0: 1})

In [388]:
Counter(subdata_bm.cause_Fall)

Counter({0.0: 170, 1.0: 77})